In [1]:
# Import required libraries
import pandas as pd
!pip install dash
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

<ipython-input-1-3202b6b80139>:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-3202b6b80139>:6: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SpaceX/Dataset/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [ ]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # Add a dropdown list to enable Launch Site selection
                                dcc.Dropdown(id='site-dropdown',
                                             options=[{'label': 'All Sites', 'value': 'ALL'},
                                                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                      {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                      {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                      {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},],
                                             value='ALL',
                                             placeholder="place holder here",
                                             searchable=True
                                ),
                                html.Br(),

                                # Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 100: '100'},
                                                value=[min_payload, max_payload]),

                                #  Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df['Launch Site'], values=filtered_df['class'],
        names='Launch Site',
        title='Total Success Launches by Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_data = filtered_df['Launch Site'] == entered_site
        fig = px.pie(filtered_data, values=filtered_df['class'],
        names='Launch Site',
        title=f'Total Success Launches for site : {entered_site}')
        return fig

# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site, payload_range):
    filtered_df = spacex_df

    if entered_site == 'ALL':
        filtered_data = filtered_df[(filtered_df['Payload Mass (kg)'] >= payload_range[0]) &
                                    (filtered_df['Payload Mass (kg)'] <= payload_range[1])]
        fig = px.scatter(filtered_data,
                         x = filtered_data['Payload Mass (kg)'],
                         y = filtered_data['class'],
                         color = filtered_data['Booster Version Category'],
                         title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_data = filtered_df[(filtered_df['Launch Site'] == entered_site) &
                                    (filtered_df['Payload Mass (kg)'] >= payload_range[0]) &
                                    (filtered_df['Payload Mass (kg)'] <= payload_range[1])]

        fig = px.scatter(filtered_data,
                         x = filtered_data['Payload Mass (kg)'],
                         y = filtered_data['class'],
                         color = filtered_data['Booster Version Category'],
                         title=f'Correlation between Payload and Success for site : {entered_site}')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()